# Capítulo III: Chains

### Ejemplo inicial en el uso de cadenas

Cada vez que se desee trabajar con una cadena, al menos, debe estar definido un prompt template e instanciado algún modelo de lenguaje

In [7]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [8]:
load_dotenv()

True

In [9]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="Cual es una buena actividad para realizar en {temporada}?",
            input_variables=["temporada"],
        )
    )

In [10]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [11]:
print(chain.run("primavera"))

Una buena actividad para realizar en primavera es hacer un picnic al aire libre. Puedes buscar un parque o jardín bonito y llevar una manta, comida y bebidas. Disfrutarás del buen clima, de la naturaleza y de una comida al aire libre. También puedes llevar juegos de mesa o actividades al aire libre como una pelota o una frisbee para divertirte aún más.


In [12]:
#Las cadenas tambien pueden ser instanciadas con un estado de "memoria"
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory #<--- golazo que veremos más adelante

In [13]:
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [14]:
conversation.run("Dime los primeros 5 números de la serie de fibonacci")

'Los primeros 5 números de la serie de Fibonacci son 0, 1, 1, 2, 3. ¿Hay algo más en lo que pueda ayudarte?'

In [15]:
conversation.run("¿Y cuál es el séptimo?") #<-- preserva el contexto de la pregunta anterior

'El séptimo número de la serie de Fibonacci es 8. ¿Hay algo más en lo que pueda ayudarte?'